In [2]:
%run utils.ipynb

import pandas as pd
import numpy as np
from tqdm import tqdm
from annoy import AnnoyIndex
import os
import warnings
from collections import defaultdict
import math
import pickle
import multitasking
import signal

multitasking.set_max_threads(10)
multitasking.set_engine('process')
signal.signal(signal.SIGINT, multitasking.killall)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 10)

warnings.filterwarnings('ignore')

In [3]:
df_qtime = pd.read_pickle('../user_data/data/qtime.pkl')
df_click = pd.read_pickle('../user_data/data/click.pkl')

In [4]:
phases = sorted(list(df_qtime['phase'].unique()))
phases

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
def cal_item_sim(df, user_col, item_col):
    # df为这个阶段的click
    # 每个user的item
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))
    
    # 每个item的user
    item_user_ = df.groupby(item_col)[user_col].agg(list).reset_index()
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))
    
    # 每个user的时间
    user_time_ = df.groupby(user_col)['time'].agg(
        lambda x: list(x)).reset_index()
    user_time_dict = dict(zip(user_time_[user_col], user_time_['time']))

    sim_item = {}
    # 遍历每个item里的user
    for item, users in tqdm(item_user_dict.items()):
        # sedefault:给字典中不存在的键赋值为默认值
        sim_item.setdefault(item, {})

        for u in users:
            # 取每个user看过的items
            items = user_item_dict[u]
            
            # 遍历所有相关item
            for relate_item in items:
                # 求出相隔时间
                loc1 = user_item_dict[u].index(item)
                loc2 = user_item_dict[u].index(relate_item)
                
                # 这一步没用到
                t1 = user_time_dict[u][loc1]
                t2 = user_time_dict[u][loc2]

                sim_item[item].setdefault(relate_item, 0)
                sim_item[item][relate_item] += 1 / \
                    (math.log(len(users)+1) * math.log(len(items)+1))

    return sim_item, user_item_dict

In [27]:
def recall(df_qtime, item_sim_list, user_item):
    data_list = []

    for user_id, query_time, item_id, phase in tqdm(df_qtime.values):
        rank = {}
        # 取出这个user的所有item 并将所有的item 时间越靠后的越放前面
        interacted_items = user_item[user_id]
        interacted_items = interacted_items[::-1]
        
        # 遍历所有items
        for loc, i in enumerate(interacted_items):
            # 取每个item的相似度 并按相似度大小倒序排序（Top500）
            for j, wij in sorted(item_sim_list[i].items(),
                                 key=lambda d: d[1],
                                 reverse=True)[0:500]:
                # 过滤掉已经看过的 计算推荐rank 越靠后的求的值越小
                if j not in interacted_items:
                    rank.setdefault(j, 0)
                    rank[j] += wij * (0.7**loc)

        sim_items = sorted(rank.items(), key=lambda d: d[1],
                           reverse=True)[:100]
        item_ids = [item[0] for item in sim_items]
        item_sim_scores = [item[1] for item in sim_items]

        df_temp = pd.DataFrame()
        df_temp['item_id'] = item_ids
        df_temp['sim_score'] = item_sim_scores
        df_temp['user_id'] = user_id
        df_temp['query_time'] = query_time
        df_temp['phase'] = phase

        if item_id == -1:
            df_temp['label'] = np.nan
        else:
            df_temp['label'] = 0
            # 预测成功的label改为1 
            df_temp.loc[df_temp['item_id'] == item_id, 'label'] = 1
        
        # 根据sim_score降序排序
        df_temp.sort_values(['sim_score'], inplace=True, ascending=False)
        df_temp = df_temp[[
            'user_id', 'phase', 'query_time', 'item_id', 'sim_score', 'label'
        ]]
        df_temp['user_id'] = df_temp['user_id'].astype('int')
        df_temp['item_id'] = df_temp['item_id'].astype('int')

        data_list.append(df_temp)

    df_data = pd.concat(data_list, sort=False)
    return df_data

In [28]:
df_data = recall(df_qtime_phase, sim_item, user_item_dict)

100%|██████████| 18505/18505 [02:27<00:00, 125.85it/s]


In [25]:
df_data.head()

,user_id,query_time,item_id,phase
24,1,0.9839419315,69359,0
214,2,0.9838837214,58621,0
524,4,0.9838849522,90818,0
612,7,0.9839401177,23436,0
753,9,0.9838944402,114268,0


In [6]:
@multitasking.task
def work(phase, force=False):
    os.makedirs('../user_data/model/recall_v2', exist_ok=True)

    if force or (
            not os.path.exists(
                '../user_data/model/recall_v2/sim_{}.pkl'.format(phase))
            or not os.path.exists(
                '../user_data/model/recall_v2/recall_{}.pkl'.format(phase))):
        # 获取当前阶段的click
        df_click_phase = df_click[df_click['phase'] == phase]
        sim_item, user_item_dict = cal_item_sim(df_click_phase, 'user_id',
                                                'item_id')

        f = open('../user_data/model/recall_v2/sim_{}.pkl'.format(phase), 'wb')
        pickle.dump(sim_item, f)
        f.close()

        # 获取当前阶段的qtime, 召回
        df_qtime_phase = df_qtime[df_qtime['phase'] == phase]
        df_data = recall(df_qtime_phase, sim_item, user_item_dict)
        df_data.to_pickle(
            '../user_data/model/recall_v2/recall_{}.pkl'.format(phase))

        print('phase {} finish'.format(phase))

In [7]:
item_sim_phase = {}
df_recall = pd.DataFrame()
val_score = np.array([0.0, 0.0, 0.0, 0.0])
force = False

for phase in phases:
    work(phase, force)

multitasking.wait_for_tasks()
print('合并任务')

for phase in phases:
    f = open('../user_data/model/recall_v2/sim_{}.pkl'.format(phase), 'rb')
    item_sim = pickle.load(f)
    f.close()

    df_data = pd.read_pickle(
        '../user_data/model/recall_v2/recall_{}.pkl'.format(phase))

    item_sim_phase[phase] = item_sim
    df_recall = df_recall.append(df_data)

    score = evaluate_scores(df_data, phase)
    val_score += score

    print('phase', phase, score)

 99%|█████████▉| 19924/20047 [03:00<00:01, 93.54it/s] 

phase 0 finish


 95%|█████████▍| 18853/19883 [03:02<00:10, 100.15it/s]

phase 1 finish


 92%|█████████▏| 17308/18821 [03:04<00:14, 104.63it/s]

...

phase 7 (0.044299701028376194, 0.10169962230615419, 0.020781061524281722, 0.059474412171507604)


100%|██████████| 19882/19882 [00:14<00:00, 1408.74it/s]


phase 8 (0.044730125896910895, 0.09698311652366455, 0.021365178652059297, 0.06052269601100413)


100%|██████████| 20047/20047 [00:14<00:00, 1407.39it/s]

phase 9 (0.04502187520089846, 0.10057392730254168, 0.022375648513518566, 0.06182544970019987)


In [8]:
# 保存相似度字典给后续使用
f = open('../user_data/model/bn_sim.pkl', 'wb')
pickle.dump(item_sim_phase, f)
f.close()

In [9]:
val_score

array([0.49572583, 1.10200045, 0.22897122, 0.62744458])

In [10]:
df_recall.sort_values(['user_id', 'phase', 'query_time'], inplace=True)
df_recall.to_pickle('../user_data/data/recall_v2.pkl')
df_recall.head()

,user_id,phase,query_time,item_id,sim_score,label
0,1,0.0,0.9839419315,87837,0.3453144185,0.0
2,1,0.0,0.9839419315,19228,0.3453144185,0.0
3,1,0.0,0.9839419315,109854,0.3453144185,0.0
4,1,0.0,0.9839419315,55738,0.3453144185,0.0
1,1,0.0,0.9839419315,91290,0.3453144185,0.0
